In [ ]:
from collections import Counter
import os

def count_images_recursive(folder):
    count = 0
    for root, _, files in os.walk(folder):
        count += len([
            f for f in files 
            if f.lower().endswith(('.jpg', '.jpeg', '.png'))
        ])
    return count

# LOCAL paths
# Assuming folders are in same directory as notebook:
root_dir = os.getcwd()   # current working dir (the folder where your .ipynb is)
real_path = os.path.join(root_dir, "Real_5k")
ai_path = os.path.join(root_dir, "AI_5k")

# Count images
real_count = count_images_recursive(real_path)
ai_count = count_images_recursive(ai_path)

# Print results
print("✅ Class Distribution:")
print(f"Class 0 (Real) : {real_count} images")
print(f"Class 1 (AI)   : {ai_count} images")

✅ Class Distribution:
Class 0 (Real) : 2142 images
Class 1 (AI)   : 2117 images


In [4]:
import os

# Collect image paths from both classes
real_images = [
    os.path.join(root, f)
    for root, _, files in os.walk(real_path)
    for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png'))
]

ai_images = [
    os.path.join(root, f)
    for root, _, files in os.walk(ai_path)
    for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png'))
]

# Combine image paths and labels
all_samples = real_images + ai_images
all_labels  = [0] * len(real_images) + [1] * len(ai_images)

# Summary
print(f" Total Samples: {len(all_samples)}")
print(f" - Real [0]: {len(real_images)}")
print(f" - AI   [1]: {len(ai_images)}")


 Total Samples: 4259
 - Real [0]: 2142
 - AI   [1]: 2117


In [5]:
from sklearn.model_selection import train_test_split


# => X_test will be exactly 10% of the total dataset
X_trainval, X_test, y_trainval, y_test = train_test_split(
    all_samples, all_labels,
    test_size=0.1,
    stratify=all_labels,
    random_state=42
)

#Split the remaining 90% into:
# => 70% Train  (0.9 × 0.778 ≈ 70%)
# => 20% Val    (0.9 × 0.222 ≈ 20%)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval,
    test_size=0.222,  # 0.222 × 90% ≈ 20% of total
    stratify=y_trainval,
    random_state=42
)


print(" Dataset split:")
print(f" - Training    : {len(X_train)} samples")   # ~70%
print(f" - Validation  : {len(X_val)} samples")     # ~20%
print(f" - Test        : {len(X_test)} samples")     # 10%


 Dataset split:
 - Training    : 2982 samples
 - Validation  : 851 samples
 - Test        : 426 samples


In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
from torch.utils.data import Dataset

class FaceDataset(Dataset):
    def __init__(self, samples, labels, transform=None):
        self.samples = samples
        self.labels = labels
        self.transform = transform
        self.face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path = self.samples[idx]
        label = self.labels[idx]
        img = cv2.imread(img_path)

        # If image can't be loaded, use a black fallback
        if img is None:
            img = np.zeros((224, 224, 3), dtype=np.uint8)
        else:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # convert for PIL use

        # Resize to 224x224
        img = cv2.resize(img, (224, 224))
        pil_img = Image.fromarray(img)

        # Apply transforms
        if self.transform:
            pil_img = self.transform(pil_img)

        return pil_img, label

In [7]:
from torch.utils.data import DataLoader
from torchvision import transforms

normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

train_tf = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(0.2, 0.2, 0.2),
    transforms.ToTensor(),
    normalize
])

test_tf = transforms.Compose([
    transforms.ToTensor(),
    normalize
])

# Create datasets
train_ds = FaceDataset(X_train, y_train, transform=train_tf)
val_ds   = FaceDataset(X_val, y_val, transform=test_tf)
test_ds  = FaceDataset(X_test, y_test, transform=test_tf)

# Create dataloaders
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=32)
test_loader  = DataLoader(test_ds, batch_size=32)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"✅ Using device: {device}")

✅ Using device: cuda


In [ ]:
# ✅ Dropout-based classification head for ViT
class MCDropoutHead(nn.Module):
    def __init__(self, in_features, num_classes=2, dropout_prob=0.4):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout_prob)
        self.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        x = self.dropout(x)
        return self.fc(x)

# ✅ ResNet50 with standard Linear head
def get_resnet():
    model = models.resnet50(weights='IMAGENET1K_V1')
    model.fc = nn.Linear(model.fc.in_features, 2)
    return model.to(device)

# ✅ ViT with MCDropout head
def get_vit():
    model = models.vit_b_16(weights='IMAGENET1K_V1')
    in_features = model.heads[0].in_features
    model.heads = nn.Sequential(MCDropoutHead(in_features, num_classes=2, dropout_prob=0.4))
    return model.to(device)


In [ ]:
# Load models
resnet = get_resnet()
vit = get_vit()

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\abura/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:05<00:00, 17.5MB/s]


Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to C:\Users\abura/.cache\torch\hub\checkpoints\vit_b_16-c867db91.pth


100%|██████████| 330M/330M [00:16<00:00, 20.9MB/s] 


In [16]:
# Train ResNet
resnet = train_model(
    resnet,
    torch.optim.Adam(resnet.parameters(), lr=1e-4),
    train_loader,
    val_loader,
    epochs=30,
    patience=4
)

[Epoch 1/30] Train Loss: 0.0907 | Train Acc: 96.51% || Val Loss: 0.1792 | Val Acc: 93.42%
[Epoch 2/30] Train Loss: 0.0591 | Train Acc: 97.79% || Val Loss: 0.1789 | Val Acc: 94.01%
[Epoch 3/30] Train Loss: 0.0526 | Train Acc: 98.09% || Val Loss: 0.1869 | Val Acc: 93.42%
[Epoch 4/30] Train Loss: 0.0285 | Train Acc: 98.99% || Val Loss: 0.1755 | Val Acc: 94.48%
[Epoch 5/30] Train Loss: 0.0467 | Train Acc: 98.39% || Val Loss: 0.1491 | Val Acc: 94.95%
[Epoch 6/30] Train Loss: 0.0352 | Train Acc: 99.03% || Val Loss: 0.1947 | Val Acc: 93.07%
[Epoch 7/30] Train Loss: 0.0496 | Train Acc: 98.16% || Val Loss: 0.1687 | Val Acc: 93.18%
[Epoch 8/30] Train Loss: 0.0402 | Train Acc: 98.69% || Val Loss: 0.1870 | Val Acc: 93.07%
[Epoch 9/30] Train Loss: 0.0603 | Train Acc: 97.85% || Val Loss: 0.1558 | Val Acc: 94.01%
⛔ Early stopping triggered.


In [17]:
# Train ViT (with MC Dropout)
vit = train_model(
    vit,
    torch.optim.Adam(vit.parameters(), lr=5e-5),
    train_loader,
    val_loader,
    epochs=30,
    patience=4
)

[Epoch 1/30] Train Loss: 0.4988 | Train Acc: 75.55% || Val Loss: 0.3790 | Val Acc: 82.26%
[Epoch 2/30] Train Loss: 0.2381 | Train Acc: 89.27% || Val Loss: 0.2282 | Val Acc: 90.72%
[Epoch 3/30] Train Loss: 0.1674 | Train Acc: 93.46% || Val Loss: 0.2928 | Val Acc: 88.84%
[Epoch 4/30] Train Loss: 0.0935 | Train Acc: 96.71% || Val Loss: 0.2455 | Val Acc: 91.07%
[Epoch 5/30] Train Loss: 0.1045 | Train Acc: 96.28% || Val Loss: 0.1628 | Val Acc: 94.12%
[Epoch 6/30] Train Loss: 0.0469 | Train Acc: 98.16% || Val Loss: 0.2482 | Val Acc: 92.01%
[Epoch 7/30] Train Loss: 0.0544 | Train Acc: 97.79% || Val Loss: 0.3271 | Val Acc: 90.25%
[Epoch 8/30] Train Loss: 0.0578 | Train Acc: 97.52% || Val Loss: 0.3396 | Val Acc: 89.07%
[Epoch 9/30] Train Loss: 0.0304 | Train Acc: 98.89% || Val Loss: 0.3252 | Val Acc: 90.83%
⛔ Early stopping triggered.


In [ ]:
# torch.save(model.state_dict(), "checkpoint_vit.pth")

In [20]:
# Save models
torch.save(resnet.state_dict(), "resnet_trained.pth")
torch.save(vit.state_dict(), "vit_trained.pth")

# (Optional) Download if on Colab
from google.colab import files
files.download("resnet_trained.pth")
files.download("vit_trained.pth")


ModuleNotFoundError: No module named 'google.colab'

In [21]:
def ensemble_predict(model1, model2, loader):
    model1.eval()
    model2.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            out1 = F.softmax(model1(imgs), dim=1)
            out2 = F.softmax(model2(imgs), dim=1)
            final = (out1 + out2) / 2
            preds = final.argmax(dim=1).cpu().numpy()

            all_preds.extend(preds)
            all_labels.extend(labels.numpy())

    return all_labels, all_preds


In [22]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import torch.nn.functional as F

# 🔍 Evaluate a single model
def evaluate_model(model, loader):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            outputs = model(imgs)
            preds = outputs.argmax(dim=1).cpu().numpy()

            all_preds.extend(preds)
            all_labels.extend(labels.numpy())

    return all_labels, all_preds

# 📊 Evaluate all splits for a model
def evaluate_all_splits(model, name="Model"):
    for title, loader in zip(["Train", "Validation", "Test"], [train_loader, val_loader, test_loader]):
        y_true, y_pred = evaluate_model(model, loader)
        acc = accuracy_score(y_true, y_pred)
        print(f"\n📊 {name} - {title}")
        print(f"Accuracy: {acc:.4f}")
        print(classification_report(y_true, y_pred, target_names=["Real", "AI"]))
        print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))

# 🤝 Soft voting ensemble
def evaluate_ensemble(model1, model2, loader):
    model1.eval()
    model2.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            out1 = F.softmax(model1(imgs), dim=1)
            out2 = F.softmax(model2(imgs), dim=1)
            final = (out1 + out2) / 2
            preds = final.argmax(dim=1).cpu().numpy()

            all_preds.extend(preds)
            all_labels.extend(labels.numpy())

    return all_labels, all_preds

# 📊 Evaluate ensemble across all splits
def evaluate_all_ensemble(model1, model2, name="Ensemble"):
    for title, loader in zip(["Train", "Validation", "Test"], [train_loader, val_loader, test_loader]):
        y_true, y_pred = evaluate_ensemble(model1, model2, loader)
        acc = accuracy_score(y_true, y_pred)
        print(f"\n🤝 {name} - {title}")
        print(f"Accuracy: {acc:.4f}")
        print(classification_report(y_true, y_pred, target_names=["Real", "AI"]))
        print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))




In [ ]:
# ✅ Run evaluations
evaluate_all_splits(resnet, "ResNet")


📊 ResNet - Train
Accuracy: 0.9899
              precision    recall  f1-score   support

        Real       0.98      1.00      0.99      1500
          AI       1.00      0.98      0.99      1482

    accuracy                           0.99      2982
   macro avg       0.99      0.99      0.99      2982
weighted avg       0.99      0.99      0.99      2982

Confusion Matrix:
 [[1496    4]
 [  26 1456]]

📊 ResNet - Validation
Accuracy: 0.9401
              precision    recall  f1-score   support

        Real       0.92      0.96      0.94       428
          AI       0.96      0.92      0.94       423

    accuracy                           0.94       851
   macro avg       0.94      0.94      0.94       851
weighted avg       0.94      0.94      0.94       851

Confusion Matrix:
 [[412  16]
 [ 35 388]]

📊 ResNet - Test
Accuracy: 0.9249
              precision    recall  f1-score   support

        Real       0.90      0.96      0.93       214
          AI       0.96      0.89      0

In [24]:
evaluate_all_splits(vit, "ViT")



📊 ViT - Train
Accuracy: 0.9869
              precision    recall  f1-score   support

        Real       1.00      0.98      0.99      1500
          AI       0.98      1.00      0.99      1482

    accuracy                           0.99      2982
   macro avg       0.99      0.99      0.99      2982
weighted avg       0.99      0.99      0.99      2982

Confusion Matrix:
 [[1464   36]
 [   3 1479]]

📊 ViT - Validation
Accuracy: 0.9083
              precision    recall  f1-score   support

        Real       0.97      0.85      0.90       428
          AI       0.86      0.97      0.91       423

    accuracy                           0.91       851
   macro avg       0.91      0.91      0.91       851
weighted avg       0.92      0.91      0.91       851

Confusion Matrix:
 [[362  66]
 [ 12 411]]

📊 ViT - Test
Accuracy: 0.8944
              precision    recall  f1-score   support

        Real       0.96      0.83      0.89       214
          AI       0.85      0.96      0.90      

In [25]:
evaluate_all_ensemble(resnet, vit, "ResNet + ViT Ensemble")


🤝 ResNet + ViT Ensemble - Train
Accuracy: 1.0000
              precision    recall  f1-score   support

        Real       1.00      1.00      1.00      1500
          AI       1.00      1.00      1.00      1482

    accuracy                           1.00      2982
   macro avg       1.00      1.00      1.00      2982
weighted avg       1.00      1.00      1.00      2982

Confusion Matrix:
 [[1500    0]
 [   0 1482]]

🤝 ResNet + ViT Ensemble - Validation
Accuracy: 0.9612
              precision    recall  f1-score   support

        Real       0.97      0.95      0.96       428
          AI       0.95      0.97      0.96       423

    accuracy                           0.96       851
   macro avg       0.96      0.96      0.96       851
weighted avg       0.96      0.96      0.96       851

Confusion Matrix:
 [[408  20]
 [ 13 410]]

🤝 ResNet + ViT Ensemble - Test
Accuracy: 0.9554
              precision    recall  f1-score   support

        Real       0.97      0.94      0.96      

In [29]:
print("📗 ResNet Test Report")
true_r, pred_r = evaluate_model(resnet, test_loader)
print(classification_report(true_r, pred_r, target_names=["Real", "AI"]))
print("Confusion Matrix:\n", confusion_matrix(true_r, pred_r))
print(f"Accuracy: {accuracy_score(true_r, pred_r):.4f}\n")

print("📘 ViT Test Report")
true_v, pred_v = evaluate_model(vit, test_loader)
print(classification_report(true_v, pred_v, target_names=["Real", "AI"]))
print("Confusion Matrix:\n", confusion_matrix(true_v, pred_v))
print(f"Accuracy: {accuracy_score(true_v, pred_v):.4f}\n")

print("🤝 Ensemble Test Report (ResNet + ViT)")
true_e, pred_e = evaluate_ensemble(resnet, vit, test_loader)
print(classification_report(true_e, pred_e, target_names=["Real", "AI"]))
print("Confusion Matrix:\n", confusion_matrix(true_e, pred_e))
print(f"Accuracy: {accuracy_score(true_e,pred_e):.4f}")

📗 ResNet Test Report
              precision    recall  f1-score   support

        Real       0.90      0.96      0.93       214
          AI       0.96      0.89      0.92       212

    accuracy                           0.92       426
   macro avg       0.93      0.92      0.92       426
weighted avg       0.93      0.92      0.92       426

Confusion Matrix:
 [[206   8]
 [ 24 188]]
Accuracy: 0.9249

📘 ViT Test Report
              precision    recall  f1-score   support

        Real       0.96      0.83      0.89       214
          AI       0.85      0.96      0.90       212

    accuracy                           0.89       426
   macro avg       0.90      0.89      0.89       426
weighted avg       0.90      0.89      0.89       426

Confusion Matrix:
 [[177  37]
 [  8 204]]
Accuracy: 0.8944

🤝 Ensemble Test Report (ResNet + ViT)
              precision    recall  f1-score   support

        Real       0.97      0.94      0.96       214
          AI       0.94      0.97      0

📗 ResNet Test Report
              precision    recall  f1-score   support

        Real       0.90      0.96      0.93       214
          AI       0.96      0.89      0.92       212

    accuracy                           0.92       426
   macro avg       0.93      0.92      0.92       426
weighted avg       0.93      0.92      0.92       426

Confusion Matrix:
 [[206   8]
 [ 24 188]]
Accuracy: 0.9249

📘 ViT Test Report
              precision    recall  f1-score   support

        Real       0.96      0.83      0.89       214
          AI       0.85      0.96      0.90       212

    accuracy                           0.89       426
   macro avg       0.90      0.89      0.89       426
weighted avg       0.90      0.89      0.89       426

Confusion Matrix:
 [[177  37]
 [  8 204]]
Accuracy: 0.8944

🤝 Ensemble Test Report (ResNet + ViT)
              precision    recall  f1-score   support

        Real       0.97      0.94      0.96       214
          AI       0.94      0.97      0.96       212

    accuracy                           0.96       426
   macro avg       0.96      0.96      0.96       426
weighted avg       0.96      0.96      0.96       426

Confusion Matrix:
 [[202  12]
 [  7 205]]
Accuracy: 0.9554